<a href="https://colab.research.google.com/github/renzotaco/IOIII/blob/main/Mapa_Distancias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ortools

     |████████████████████████████████| 14.4 MB 74 kB/s 


**importamos librerias**

In [2]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from __future__ import print_function
from ortools.graph import pywrapgraph #Librería de Google Or Tools

%matplotlib inline

In [11]:
df = pd.read_excel('LugaresDistancias.xlsx')
df_1 = pd.read_excel('LugaresDistancias.xlsx', sheet_name=1) #Función que permite leer una hoja en específico (hoja 2 de Excel - Inicia desde 0)

data = {} #Crea un directorio llamado data, en él agregaremos cada lista de datos junto con su nombre (índice)
data['fuentes'] = df['fuentes'].tolist() #Columna en Excel = 'fuentes' > lista en el directorio "data" con la etiqueta (índice) "fuentes"
data['destinos'] = df['destinos'].tolist() #Columna en Excel = 'destinos' > lista en el directorio "data" con la etiqueta (índice) "destinos"
data['capacidad'] = df['capacidad'].tolist() #Columna en Excel = 'capacidad' > lista en el directorio "data" con la etiqueta (índice) "capacidad"
data['distancias'] = df['distancia'].tolist() #Columna en Excel = 'distancia' > lista en el directorio "data" con la etiqueta (índice) "distancia"
data['suministro'] = df_1['suministros'].tolist() #Columna en Excel = 'suministro' > lista en el directorio "data" con la etiqueta (índice) "suministro"

data = create_data() #Ahora el "data" de esta función, contiene el directorio de la función "create_data". Ya podemos usar los listados.

# Crea una instancia para el solucionador
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

# Define cada arco del problema
for i in range(0, len(data['fuentes'])):
  min_cost_flow.AddArcWithCapacityAndUnitCost(data['fuentes'][i], data['destinos'][i],
                                              data['capacidad'][i], data['distancias'][i])
# Define los suministros para cada nodo.
for i in range(0, len(data['suministro'])):
  min_cost_flow.SetNodeSupply(i, data['suministro'][i])

# Encuentra el costo mínimo entre el nodo 0 y el nodo 8
if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
  print('Distancia mínima:', min_cost_flow.OptimalCost())
  print('')
  print('  Arco    Flujo / Capacidad  Distancia')
  for i in range(min_cost_flow.NumArcs()):
    cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
    print('%1s -> %1s    %3s   / %3s       %3s' % (
        min_cost_flow.Tail(i),
        min_cost_flow.Head(i),
        min_cost_flow.Flow(i),
        min_cost_flow.Capacity(i),
        cost))
else:
  print('Hubo un problema con la entrada de flujo de distancia mínima.')

Distancia mínima: 7

  Arco    Flujo / Capacidad  Distancia
0 -> 1      1   /   1         2
0 -> 2      0   /   1         0
0 -> 3      0   /   1         0
1 -> 0      0   /   1         0
1 -> 2      0   /   1         0
1 -> 4      1   /   1         3
2 -> 0      0   /   1         0
2 -> 1      0   /   1         0
2 -> 3      0   /   1         0
2 -> 4      0   /   1         0
2 -> 5      0   /   1         0
3 -> 0      0   /   1         0
3 -> 2      0   /   1         0
3 -> 4      0   /   1         0
3 -> 5      0   /   1         0
3 -> 6      0   /   1         0
4 -> 1      0   /   1         0
4 -> 2      0   /   1         0
4 -> 3      0   /   1         0
4 -> 5      0   /   1         0
4 -> 6      1   /   1         2
5 -> 2      0   /   1         0
5 -> 3      0   /   1         0
5 -> 4      0   /   1         0
5 -> 6      0   /   1         0
6 -> 3      0   /   1         0
6 -> 4      0   /   1         0
6 -> 5      0   /   1         0


In [12]:
df.head()

,fuentes,destinos,capacidad,distancia,Latitud,Longitud,Lugar,Icono,Nodos
0,0,1,1,2,-18.037621,-70.250694,Perez Gamboa,cloud,0.0
1,0,2,1,1,-18.032867,-70.251729,Mercado Cenepa,cloud,1.0
2,0,3,1,4,-18.027787,-70.251058,Ovalo Cusco,cloud,2.0
3,1,0,1,2,-18.024660,-70.251445,Universidad Nacional,cloud,3.0
4,1,2,1,6,-18.018023,-70.253054,Plaza Vea,cloud,4.0


In [14]:
m = folium.Map([-18.039851567978463, -70.25436292330609],  zoom_start=14)

for index, row in df.iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  popup=row['Lugar'],
                  icon=folium.Icon(icon='Icono')
                 ).add_to(m)
m